In [1]:

import pandas as pd
import requests
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
data = pd.read_csv("files/marinedata.csv", index_col=0)

In [3]:
data.columns

Index(['date_year', 'sampleSizeUnit', 'class', 'scientificName', 'depth',
       'institutionCode', 'shoredistance', 'flags', 'maximumDepthInMeters',
       'absence', 'date_start', 'minimumDepthInMeters', 'subclass',
       'basisOfRecord', 'sss', 'sex', 'organismQuantity', 'sst', 'family',
       'kingdom', 'occurrenceID', 'bathymetry', 'date_end',
       'organismQuantityType', 'species', 'sampleSizeValue', 'genus',
       'waterBody', 'vernacularName', 'id_node', 'name', 'lon', 'lat'],
      dtype='object')

In [ ]:
data["date_start"] = pd.to_datetime(data["date_start"], unit='ms')
data.drop('absence', axis= 1, inplace= True)
data.drop('sampleSizeValue', axis= 1, inplace= True)


In [13]:
geolocator = Nominatim(user_agent="IreneDHA")

In [14]:
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [17]:
def get_city_country(row):
    lat = row['lat']
    lon = row['lon']

    try:
    # geopy.reverse recibe (lat, lon)
        location = reverse((lat, lon), language='en', exactly_one=True)
        if location is None:
            return pd.Series([None, None])
        address = location.raw.get('address', {}) # la respuesta cruda contiene un dict 'address'
        # diferentes sitios usan claves distintas para "ciudad"
        city = address.get('city') or address.get('town') or address.get('village') or address.get('municipality') or address.get('hamlet')
        country = address.get('country')
        return pd.Series([city, country])
    except Exception:
        # en caso de error (timeout, servidor) devolvemos None para no romper el proceso
        return pd.Series([None, None])


In [18]:
data[['city', 'country']] = data.apply(get_city_country, axis=1)

In [19]:
data.columns

Index(['date_year', 'sampleSizeUnit', 'class', 'scientificName', 'depth',
       'institutionCode', 'shoredistance', 'flags', 'maximumDepthInMeters',
       'date_start', 'minimumDepthInMeters', 'subclass', 'basisOfRecord',
       'sss', 'sex', 'organismQuantity', 'sst', 'family', 'kingdom',
       'occurrenceID', 'bathymetry', 'date_end', 'organismQuantityType',
       'species', 'genus', 'waterBody', 'vernacularName', 'id_node', 'name',
       'lon', 'lat', 'city', 'country'],
      dtype='object')

In [20]:
data['country'].value_counts()

country
United States     217
United Kingdom    118
Belgium           111
Australia          43
New Zealand        12
Greece             10
India               5
Canada              1
Ukraine             1
Ecuador             1
Name: count, dtype: int64

In [21]:
data['country'].isna().sum()

0

In [22]:
data['city'].value_counts()

city
Durham                   183
Plymouth                 118
Ostend                   101
Hobart                    43
Boulder                   34
Waitematā                 12
Gouves Municipal Unit     10
Brussels                  10
Ernakulam                  5
Dartmouth                  1
Odesa                      1
Guayaquil                  1
Name: count, dtype: int64

In [23]:
data['city'].isna().sum()

0

In [29]:
vernacular_corrections = {
    'Atlantic gray seal': 'Grey seal',
    'humpback whale': 'Humpback whale', 
    'WHALE-HUMPBACK': 'Humpback whale',
    'Humpback Whale': 'Humpback whale',
    'Humpback': 'Humpback whale',
    'Common Bottlenose Dolphin': 'Bottlenose dolphin',
    'Bottlenose dolphin': 'Bottlenose dolphin',
    'Cheilinus undulatus': 'Napoleon fish',
    'Humphead wrasse': 'Napoleon fish',
    }

In [30]:
data['vernacularName'] = data['vernacularName'].replace(vernacular_corrections)


In [32]:
data['scientificName'].value_counts(dropna=False)

scientificName
Neoturris pileata         100
Megaptera novaeangliae    100
Cheilinus undulatus       100
Tursiops truncatus        100
Halichoerus grypus        100
Phycocaris simulans        19
Name: count, dtype: int64

In [25]:
data['scientificName'].unique()

array(['Neoturris pileata', 'Megaptera novaeangliae',
       'Phycocaris simulans', 'Cheilinus undulatus', 'Tursiops truncatus',
       'Halichoerus grypus'], dtype=object)

In [33]:
dic = {'Neoturris pileata': 'Pileata jellyfish', 'Phycocaris simulans': 'Hairy shrimp', 'Megaptera novaeangliae': 'Humpback whale', 'Cheilinus undulatus':'Napoleon fish' , 'Tursiops truncatus': 'Bottlenose dolphin', 'Halichoerus grypus': 'Grey seal'}

In [34]:

data["vernacularName"] = data["vernacularName"].fillna(data["scientificName"].map(dic))


In [35]:
data['vernacularName'].value_counts(dropna=False)

vernacularName
Pileata jellyfish     100
Humpback whale        100
Napoleon fish         100
Bottlenose dolphin    100
Grey seal             100
Hairy shrimp           19
Name: count, dtype: int64

In [36]:
data["date_end"] = pd.to_datetime(data["date_end"], unit='ms')
data.drop('organismQuantity', axis= 1, inplace= True)
data.drop('organismQuantityType', axis= 1, inplace= True)


In [37]:

data['date_year'] = data['date_year'].astype('Int64')


In [43]:
data["flags"] = data["flags"].replace("[]", "sin observaciones")
data["institutionCode"] = data["institutionCode"].fillna("sin datos")
data["sampleSizeUnit"] = data["sampleSizeUnit"].fillna("desconocido")
data["bathymetry"] = data["bathymetry"].replace("-", "")

In [44]:
data.drop('depth', axis= 1, inplace= True)

In [45]:
data.columns = (
    data.columns
    .str.strip()        # quita espacios al inicio y fin
    .str.lower()        # pasa a minúsculas
    .str.replace(" ", "_")  # reemplaza espacios por guión bajo
)

In [46]:
data.columns

Index(['date_year', 'samplesizeunit', 'class', 'scientificname',
       'institutioncode', 'shoredistance', 'flags', 'maximumdepthinmeters',
       'date_start', 'minimumdepthinmeters', 'subclass', 'basisofrecord',
       'sss', 'sex', 'sst', 'family', 'kingdom', 'occurrenceid', 'bathymetry',
       'date_end', 'species', 'genus', 'waterbody', 'vernacularname',
       'id_node', 'name', 'lon', 'lat', 'city', 'country'],
      dtype='object')

In [41]:
data['depth'].value_counts(dropna=False)

depth
NaN        257
0.00        95
10.00       83
5.00        20
4.00         3
23.20        2
8.00         2
15.10        2
29.00        2
23.00        2
20.00        2
16.60        1
12.00        1
8.40         1
16.50        1
24.00        1
14.25        1
15.50        1
9.00         1
17.29        1
7.00         1
14.45        1
21.50        1
15.00        1
17.59        1
13.83        1
15.86        1
0.50         1
9.95         1
17.28        1
17.60        1
9.50         1
13.38        1
0.61         1
19.31        1
16.52        1
3952.00      1
11.50        1
26.00        1
28.00        1
20.05        1
14.75        1
0.92         1
11.17        1
55.00        1
17.20        1
15.40        1
15.55        1
9.20         1
9.25         1
14.00        1
12.74        1
13.00        1
21.30        1
16.66        1
18.00        1
1.37         1
21.00        1
12.70        1
22.94        1
Name: count, dtype: int64

In [39]:
data['bathymetry'].value_counts(dropna=False)

bathymetry
51.00     84
2.00      20
-1.00     17
6.00       9
56.00      9
          ..
1.56       1
-72.00     1
3.26       1
3.46       1
57.00      1
Name: count, Length: 244, dtype: int64

In [ ]:
data.to_csv('files/marinedata_final.csv', index=False)